In [1]:
import torch
from torch.autograd import Variable
from gpytorch.utils import LanczosBidiagonalize, StochasticLQ
from gpytorch.kernels import RBFKernel

In [2]:
N = 500
max_iter = 50

kern = RBFKernel(log_lengthscale_bounds=(1, 1))
X = torch.linspace(0, 1, N)
var_X = Variable(X)
rhs_vec = torch.randn(N, 1)

In [3]:
K = kern(var_X, var_X)

In [4]:
K = (K + Variable(torch.diag(1e-4 * torch.ones(len(var_X))))).data

In [5]:
P, B, V = LanczosBidiagonalize(max_iter=500).lanczos_bidiagonalize(lambda v: K.matmul(v), rhs_vec)

In [6]:
Qs, Ts = StochasticLQ(max_iter=15).lanczos_batch(lambda v: K.matmul(v), rhs_vec)
Q = Qs[0]
T = Ts[0]

In [7]:
Q.t().matmul(Q)


 1.0000  0.0000 -0.0000  0.0000 -0.0000 -0.0000 -0.0000
 0.0000  1.0000  0.0000 -0.0000 -0.0000 -0.0000 -0.0000
-0.0000  0.0000  1.0000 -0.0000 -0.0000  0.0000  0.0000
 0.0000 -0.0000 -0.0000  1.0000  0.0000  0.0000  0.0000
-0.0000 -0.0000 -0.0000  0.0000  1.0000 -0.0000  0.0000
-0.0000 -0.0000  0.0000  0.0000 -0.0000  1.0000  0.0000
-0.0000 -0.0000  0.0000 -0.0000  0.0000  0.0000  1.0000
[torch.FloatTensor of size 7x7]

In [8]:
V.t().matmul(V)


 1.0000  0.0000 -0.0000  ...  -0.0000  0.0000  0.0000
 0.0000  1.0000  0.0000  ...  -0.0000 -0.0000  0.0000
-0.0000  0.0000  1.0000  ...  -0.0000  0.0000  0.0000
          ...             ⋱             ...          
-0.0000  0.0000  0.0000  ...   1.0000  0.0000 -0.0000
-0.0000 -0.0000  0.0000  ...   0.0000  1.0000  0.0000
-0.0000  0.0000  0.0000  ...  -0.0000  0.0000  1.0000
[torch.FloatTensor of size 500x500]

In [13]:
torch.svd(K)[1]


 4.7166e+02
 2.7638e+01
 6.8806e-01
 1.1015e-02
 2.2784e-04
 1.3527e-04
 1.3175e-04
 1.2916e-04
 1.2460e-04
 1.1905e-04
 1.1577e-04
 1.1366e-04
 1.1120e-04
 1.1040e-04
 1.0751e-04
 1.0690e-04
 1.0661e-04
 1.0573e-04
 1.0550e-04
 1.0484e-04
 1.0437e-04
 1.0362e-04
 1.0336e-04
 1.0329e-04
 1.0322e-04
 1.0316e-04
 1.0303e-04
 1.0298e-04
 1.0292e-04
 1.0287e-04
 1.0282e-04
 1.0274e-04
 1.0271e-04
 1.0268e-04
 1.0265e-04
 1.0260e-04
 1.0257e-04
 1.0255e-04
 1.0253e-04
 1.0249e-04
 1.0245e-04
 1.0242e-04
 1.0239e-04
 1.0237e-04
 1.0235e-04
 1.0233e-04
 1.0230e-04
 1.0227e-04
 1.0225e-04
 1.0223e-04
 1.0222e-04
 1.0220e-04
 1.0218e-04
 1.0215e-04
 1.0213e-04
 1.0211e-04
 1.0209e-04
 1.0207e-04
 1.0204e-04
 1.0203e-04
 1.0201e-04
 1.0197e-04
 1.0196e-04
 1.0194e-04
 1.0193e-04
 1.0192e-04
 1.0189e-04
 1.0188e-04
 1.0186e-04
 1.0186e-04
 1.0183e-04
 1.0183e-04
 1.0181e-04
 1.0177e-04
 1.0175e-04
 1.0174e-04
 1.0173e-04
 1.0172e-04
 1.0171e-04
 1.0170e-04
 1.0168e-04
 1.0167e-04
 1.0164e-04
 1.

In [9]:
test_vec = torch.randn(N)
test_vec = test_vec / torch.norm(test_vec)

In [10]:
torch.norm(P.matmul(B).matmul(V.t().matmul(test_vec)) - K.matmul(test_vec))

4.6171483005901385e-06

In [29]:
torch.norm(Q.matmul(T.matmul(Q.t().matmul(test_vec))) - K.matmul(test_vec))

9.98176422058089e-05

In [11]:
Q.matmul(T.matmul(Q.t().matmul(test_vec)))[:5]


1.00000e-03 *
  7.6811
  7.6705
  7.6597
  7.6499
  7.6404
[torch.FloatTensor of size 5]

In [19]:
P.matmul(B).matmul(V.t().matmul(test_vec))[:5]


-0.4679
-0.4682
-0.4686
-0.4690
-0.4693
[torch.FloatTensor of size 5]

In [18]:
K.matmul(test_vec)[:5]


-0.4679
-0.4683
-0.4686
-0.4689
-0.4693
[torch.FloatTensor of size 5]

In [32]:
P.t().matmul(P)



Columns 0 to 5 
 1.0000e+00 -6.7055e-08  5.9605e-08  1.4901e-08  5.5879e-08 -1.1057e-05
-6.7055e-08  1.0000e+00  8.1956e-08  7.4506e-09  1.6065e-07 -2.2037e-05
 5.9605e-08  8.1956e-08  1.0000e+00  2.6822e-07 -4.8429e-08  7.8976e-06
 1.4901e-08  7.4506e-09  2.6822e-07  1.0000e+00 -2.2352e-08 -2.9311e-05
 5.5879e-08  1.6065e-07 -4.8429e-08 -2.2352e-08  1.0000e+00  1.1157e-04
-1.1057e-05 -2.2037e-05  7.8976e-06 -2.9311e-05  1.1157e-04  1.0000e+00
 4.0631e-04  1.9263e-03 -1.7542e-04 -2.4059e-04 -4.6696e-04  6.3809e-02
 6.8763e-04  3.5326e-03 -3.3697e-04 -3.8305e-04 -7.4591e-04  1.0382e-01
 7.8510e-04  4.0080e-03 -3.6802e-04 -4.5680e-04 -8.6511e-04  1.2026e-01
 8.5121e-04  4.3227e-03 -4.0093e-04 -4.8507e-04 -9.2720e-04  1.2919e-01

Columns 6 to 9 
 4.0631e-04  6.8763e-04  7.8510e-04  8.5121e-04
 1.9263e-03  3.5326e-03  4.0080e-03  4.3227e-03
-1.7542e-04 -3.3697e-04 -3.6802e-04 -4.0093e-04
-2.4059e-04 -3.8305e-04 -4.5680e-04 -4.8507e-04
-4.6696e-04 -7.4591e-04 -8.6511e-04 -9.2720e-04
 6.38

In [34]:
norms = P[:, :-1].t().matmul(P[:, -1])

In [35]:
norms


 0.0009
 0.0043
-0.0004
-0.0005
-0.0009
 0.1292
 0.9978
 0.9997
 1.0000
[torch.FloatTensor of size 9]

In [38]:
P[:, -1] = P[:, -1] - torch.sum(norms * P[:, :-1], dim=1)

In [39]:
P.t().matmul(P)



Columns 0 to 5 
 1.0000e+00 -6.7055e-08  5.9605e-08  1.4901e-08  5.5879e-08 -1.1057e-05
-6.7055e-08  1.0000e+00  8.1956e-08  7.4506e-09  1.6065e-07 -2.2037e-05
 5.9605e-08  8.1956e-08  1.0000e+00  2.6822e-07 -4.8429e-08  7.8976e-06
 1.4901e-08  7.4506e-09  2.6822e-07  1.0000e+00 -2.2352e-08 -2.9311e-05
 5.5879e-08  1.6065e-07 -4.8429e-08 -2.2352e-08  1.0000e+00  1.1157e-04
-1.1057e-05 -2.2037e-05  7.8976e-06 -2.9311e-05  1.1157e-04  1.0000e+00
 4.0631e-04  1.9263e-03 -1.7542e-04 -2.4059e-04 -4.6696e-04  6.3809e-02
 6.8763e-04  3.5326e-03 -3.3697e-04 -3.8305e-04 -7.4591e-04  1.0382e-01
 7.8510e-04  4.0080e-03 -3.6802e-04 -4.5680e-04 -8.6511e-04  1.2026e-01
-1.8765e-03 -9.4586e-03  8.7882e-04  1.0835e-03  2.0623e-03 -2.8771e-01

Columns 6 to 9 
 4.0631e-04  6.8763e-04  7.8510e-04 -1.8765e-03
 1.9263e-03  3.5326e-03  4.0080e-03 -9.4586e-03
-1.7542e-04 -3.3697e-04 -3.6802e-04  8.7882e-04
-2.4059e-04 -3.8305e-04 -4.5680e-04  1.0835e-03
-4.6696e-04 -7.4591e-04 -8.6511e-04  2.0623e-03
 6.38

In [45]:
any(P.t().matmul(P)[:4, 5] > 1e-5)

False